# imports and loading DataFrame

In [58]:
import seaborn as sns
import pandas as pd
import datetime

DATE_FORMAT = "%Y-%m-%dT%H:%M:%S"


# fact table
sessions_df = pd.read_json("data/sessions.jsonl", lines=True)

# dimension tables
deliveries_df = pd.read_json("data/deliveries.jsonl", lines=True)
products_df = pd.read_json("data/products.jsonl", lines=True)
users_df = pd.read_json("data/users.jsonl", lines=True)

# adding a column with time difference in deliveries
adding a column with time difference between purchase_timestamp and delivery_timestamp in deliveries table

1. Cut microseconds from delivery_timestamp, so it will be the same format as purchase_timestamp, because there are no microseconds in purchase_timestamp (using "." as a separator).
2. Change columns format to datetime
3. Add time_diff column (as timedelta64 object).
4. Drop rows where time_diff is null (which means that delivery_timestamp was null).
5. Change type of time_diff from timedelta64 to seconds in float.
6. Drop rows where time_diff is below 0.

In [59]:
# 1.
deliveries_df["delivery_timestamp"] = deliveries_df["delivery_timestamp"].str.split('.', expand=True)[0]

# 2.
deliveries_df["purchase_timestamp"] = pd.to_datetime(deliveries_df["purchase_timestamp"], format=DATE_FORMAT)
deliveries_df["delivery_timestamp"] = pd.to_datetime(deliveries_df["delivery_timestamp"], format=DATE_FORMAT)

# 3.
deliveries_df["time_diff"] = deliveries_df["delivery_timestamp"] - deliveries_df["purchase_timestamp"]

# 4.
deliveries_df = deliveries_df[deliveries_df["time_diff"].notna()]

# 5.
# time diff as duration in seconds
deliveries_df["time_diff"] = deliveries_df["time_diff"].apply(datetime.timedelta.total_seconds)

# 6.
deliveries_df = deliveries_df[deliveries_df["time_diff"] >= 0]


deliveries_df.info()
deliveries_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3956 entries, 0 to 6093
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   purchase_id         3956 non-null   int64         
 1   purchase_timestamp  3956 non-null   datetime64[ns]
 2   delivery_timestamp  3956 non-null   datetime64[ns]
 3   delivery_company    3743 non-null   float64       
 4   time_diff           3956 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int64(1)
memory usage: 185.4 KB


,purchase_id,purchase_timestamp,delivery_timestamp,delivery_company,time_diff
0,20001,2022-03-04 12:00:32,2022-03-05 04:43:47,516.0,60195.0
2,20003,2022-02-23 01:05:44,2022-02-24 08:09:17,620.0,111813.0
3,20004,2022-01-12 19:51:01,2022-01-15 16:13:50,360.0,246169.0
4,20005,2022-03-16 20:50:14,2022-03-20 08:42:59,516.0,301965.0
9,20010,2022-02-28 21:15:35,2022-03-01 13:58:00,360.0,60145.0


# join deliveries with sessions

In [60]:
# drop rows where event_type is not equal "BUY_PRODUCT"
sessions_df = sessions_df[sessions_df["event_type"] == "BUY_PRODUCT"]
df = deliveries_df.merge(sessions_df, on="purchase_id", how="left")

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3956 entries, 0 to 3955
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   purchase_id         3956 non-null   int64         
 1   purchase_timestamp  3956 non-null   datetime64[ns]
 2   delivery_timestamp  3956 non-null   datetime64[ns]
 3   delivery_company    3743 non-null   float64       
 4   time_diff           3956 non-null   float64       
 5   session_id          3956 non-null   int64         
 6   timestamp           3956 non-null   datetime64[ns]
 7   user_id             3755 non-null   float64       
 8   product_id          3754 non-null   float64       
 9   event_type          3956 non-null   object        
 10  offered_discount    3956 non-null   int64         
dtypes: datetime64[ns](3), float64(4), int64(3), object(1)
memory usage: 370.9+ KB


,purchase_id,purchase_timestamp,delivery_timestamp,delivery_company,time_diff,session_id,timestamp,user_id,product_id,event_type,offered_discount
0,20001,2022-03-04 12:00:32,2022-03-05 04:43:47,516.0,60195.0,124,2022-03-04 12:00:32,102.0,1432.0,BUY_PRODUCT,15
1,20003,2022-02-23 01:05:44,2022-02-24 08:09:17,620.0,111813.0,128,2022-02-23 01:05:44,102.0,1095.0,BUY_PRODUCT,20
2,20004,2022-01-12 19:51:01,2022-01-15 16:13:50,360.0,246169.0,129,2022-01-12 19:51:01,102.0,1169.0,BUY_PRODUCT,20
3,20005,2022-03-16 20:50:14,2022-03-20 08:42:59,516.0,301965.0,130,2022-03-16 20:50:14,102.0,1433.0,BUY_PRODUCT,5
4,20010,2022-02-28 21:15:35,2022-03-01 13:58:00,360.0,60145.0,140,2022-02-28 21:15:35,102.0,1653.0,BUY_PRODUCT,5
